In [1]:
'''
Example: Supervised learning with the Naive Bayes algorithm.
'''

'\nExample: Supervised learning with the Naive Bayes algorithm.\n'

In [22]:
from __future__ import print_function

import numpy as np

import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.compare import Exact, String

In [23]:
'''
Generated as one data set with 10000 records (5000 originals and 5000 duplicates, with one duplicate per original), 
the originals have been split from the duplicates, into dataset4a.csv (containing the 5000 original records) 
and dataset4b.csv (containing the 5000 duplicate records). 
These two data sets can be used for testing linkage procedures.
'''

from recordlinkage.datasets import load_febrl4

In [24]:
# set logging
rl.logging.set_verbosity(rl.logging.INFO)

In [25]:
# load datasets
print('Loading data...')
dfA, dfB, true_links = load_febrl4(return_links=True)
print(len(dfA), 'records in dataset A')
print(len(dfB), 'records in dataset B')
print(len(true_links), 'links between dataset A and B')

Loading data...
5000 records in dataset A
5000 records in dataset B
5000 links between dataset A and B


In [27]:
# start indexing
print('Build index...')
indexer = rl.Index()
indexer.add(Block('given_name'))
indexer.add(Block('surname'))
indexer.add(Block('soc_sec_id'))
candidate_links = indexer.index(dfA, dfB)
print(len(candidate_links), 'candidate links between dataset A and B')

Build index...
INFO:recordlinkage:indexing - initialize Index class
INFO:recordlinkage:indexing [1/?] - time: 0.89s - pairs: 160170/25000000 - rr: 0.99359
INFO:recordlinkage:indexing [1/?] - time: 0.89s - pairs_total: 160170/25000000 - rr_total: 0.99359
160170 candidate links between dataset A and B


In [28]:
# start comparing
print('Start comparing...')
comparer = rl.Compare()
comparer.add(Exact('given_name', 'given_name', label='given_name'))
comparer.add(String('surname', 'surname', method='jarowinkler',
                    threshold=0.85, label='surname'))
comparer.add(Exact('date_of_birth', 'date_of_birth', label='date_of_birth'))
comparer.add(Exact('suburb', 'suburb', label='suburb'))
comparer.add(Exact('state', 'state', label='state'))
comparer.add(String('address_1', 'address_1', threshold=0.85,
                    label='address_1'))
comparer.add(String('address_2', 'address_2', threshold=0.85,
                    label='address_2'))
features = comparer.compute(candidate_links, dfA, dfB)

print('feature shape', features.shape)

Start comparing...
INFO:recordlinkage:comparing - initialize Compare class
INFO:recordlinkage:comparing [1/?] - time: 2.83s - pairs: 160170
INFO:recordlinkage:comparing [1/?] - time: 2.83s - pairs_total: 160170
feature shape (160170, 7)


In [29]:
# Initialise the NaiveBayesClassifier.
cl = rl.NaiveBayesClassifier()
cl.fit(features, true_links)

INFO:recordlinkage:Classification - start training NaiveBayesClassifier
INFO:recordlinkage:Classification - training computation time: ~0.43s


In [30]:
# Print the parameters that are trained (m, u and p). Note that the estimates
# are very good.
print("p probability P(Match):", cl.p)
print("m probabilities P(x_i=1|Match):", cl.m_probs)
print("u probabilities P(x_i=1|Non-Match):", cl.u_probs)
print("log m probabilities P(x_i=1|Match):", cl.log_m_probs)
print("log u probabilities P(x_i=1|Non-Match):", cl.log_u_probs)
print("log weights of features:", cl.log_weights)
print("weights of features:", cl.weights)

p probability P(Match): 0.030873446962602254
m probabilities P(x_i=1|Match): {'given_name': {0.0: 0.3352881765303065, 1.0: 0.6647118234696942}, 'surname': {0.0: 0.14155714290972135, 1.0: 0.85844285709028}, 'date_of_birth': {0.0: 0.106774535620848, 1.0: 0.8932254643791522}, 'suburb': {0.0: 0.2554095144424869, 1.0: 0.744590485557514}, 'state': {0.0: 0.05844288944619256, 1.0: 0.9415571105538089}, 'address_1': {0.0: 0.14034379614383036, 1.0: 0.8596562038561705}, 'address_2': {0.0: 0.3150657304321244, 1.0: 0.6849342695678765}}
u probabilities P(x_i=1|Non-Match): {'given_name': {0.0: 0.5235174746032958, 1.0: 0.4764825253967045}, 'surname': {0.0: 0.4738862941228717, 1.0: 0.5261137058771285}, 'date_of_birth': {0.0: 0.999974230310873, 1.0: 2.576968912769244e-05}, 'suburb': {0.0: 0.9991109673066697, 1.0: 0.0008890326933302852}, 'state': {0.0: 0.781890803308886, 1.0: 0.21810919669111387}, 'address_1': {0.0: 0.9994910607189673, 1.0: 0.0005089392810321288}, 'address_2': {0.0: 0.9995490410699971, 1.

In [31]:
# evaluate the model
links = cl.predict(features)
print("Predicted number of links:", len(links))

INFO:recordlinkage:Classification - predict matches and non-matches
Predicted number of links: 4923


In [32]:
cm = rl.confusion_matrix(true_links, links, len(candidate_links))
print("Confusion matrix:\n", cm)

Confusion matrix:
 [[  4915     85]
 [     8 155162]]


In [33]:
# compute the F-score for this classification
fscore = rl.fscore(cm)
print('fscore', fscore)
recall = rl.recall(true_links, links)
print('recall', recall)
precision = rl.precision(true_links, links)
print('precision', precision)

fscore 0.9906278343242971
recall 0.983
precision 0.9983749746089783


In [34]:
# Predict the match probability for each pair in the dataset.
probs = cl.prob(features)

INFO:recordlinkage:Classification - compute probabilities
